# Image Classification

In [ ]:
import arcpy
from arcpy import env
from arcpy.sa import *
env.workspace = "C:/Users/jclav/Project NRS 404/Images"

#Composite Image Bands
#Compose multi types of single band raster datasets to a TIFF format raster dataset
arcpy.CompositeBands_management("band1.tif;band2.tif;band3.tif;band4.tif;band5.tif","compbands2.tif")

#Clip Image
env.workspace = "C:/Users/jclav/Project NRS 404/Images"
arcpy.Clip_management("compbands2.tif", "#", "clip2_Q.tif","Q_BND.shp", "0", "ClippingGeometry")

#Segmetation
env.workspace = "C:/Users/jclav/Project NRS 404/Images/" 
# Setup variables
inRaster = "C:/Users/jclav/Project NRS 404/Images/clip2_Q.tif"
spectral_detail = "19"
spatial_detail = "16"
min_segment_size = "500"
band_indexes = "5 3 2"
# Execute 
seg_raster = SegmentMeanShift(inRaster, spectral_detail, spatial_detail, min_segment_size, band_indexes)
# Save the output 
seg_raster.save("C:/Users/jclav/Project NRS 404/Images/new_seg.tif")

#Random Trees Classifier
# Setup variables
inSegRaster = "C:/Users/jclav/Project NRS 404/Images/THIS_seg.tif"
train_features = "C:/Users/jclav/Project NRS 404/Images/DONE_train.shp"
out_definition = "C:/Users/jclav/Project NRS 404/Images/RT2def_sig.ecd"
in_additional_raster = "C:/Users/jclav/Project NRS 404/Images/THISclip_Q.tif"
#Since using segmented raster max sample class is zero
maxNumTrees = "50"
maxTreeDepth = "30"
maxSampleClass = "0"
attributes = "COLOR;MEAN;STD;COUNT;COMPACTNESS;RECTANGULARITY"
# Execute
TrainRandomTreesClassifier(inSegRaster, train_features,
                           out_definition, in_additional_raster, maxNumTrees,
                           maxTreeDepth, maxSampleClass, attributes)
#Classify RT
env.workspace = "C:/Users/jclav/Project NRS 404/Images" 
# Set local variables
insegras = "C:/Users/jclav/Project NRS 404/Images/THIS_seg.tif"
indef_file = "C:/Users/jclav/Project NRS 404/Images/RT2def_sig.ecd"
in_additional_raster = "C:/Users/jclav/Project NRS 404/Images/THISclip_Q.tif"
# Execute 
classifiedraster = ClassifyRaster(insegras, indef_file, in_additional_raster)
#save output
classifiedraster.save("C:/Users/jclav/Project NRS 404/Images/Q_RT2_classified.tif")


#SVM
# Set local variables
inSegRaster = "C:/Users/jclav/Project NRS 404/Images/THIS_seg.tif"
train_features = "C:/Users/jclav/Project NRS 404/Images/DONE_train.shp"
out_definition = "C:/Users/jclav/Project NRS 404/Images/SVMdef_sig.ecd"
in_additional_raster = "C:/Users/jclav/Project NRS 404/Images/THISclip_Q.tif"
maxNumSamples = "500"
attributes = "COLOR;MEAN;STD;COUNT;COMPACTNESS;RECTANGULARITY"
#Execute
arcpy.gp.TrainSupportVectorMachineClassifier(
    inSegRaster, train_features, out_definition, 
    in_additional_raster, maxNumSamples, attributes)

#Classify SVM
env.workspace = "C:/Users/jclav/Project NRS 404/Images" 
# Set local variables
insegras = "C:/Users/jclav/Project NRS 404/Images/THIS_seg.tif"
indef_file = "C:/Users/jclav/Project NRS 404/Images/SVMdef_sig.ecd"
in_additional_raster = "C:/Users/jclav/Project NRS 404/Images/THISclip_Q.tif"
# Execute 
classifiedraster = ClassifyRaster(insegras, indef_file, in_additional_raster)
#save output
classifiedraster.save("C:/Users/jclav/Project NRS 404/Images/Q_SVM_classified.tif")

#ML
# Set local variables
inSegRaster = "C:/Users/jclav/Project NRS 404/Images/THIS_seg.tif"
train_features = "C:/Users/jclav/Project NRS 404/Images/DONE_train.shp"
out_definition = "C:/Users/jclav/Project NRS 404/Images/MLdef_sig.ecd"
in_additional_raster = "C:/Users/jclav/Project NRS 404/Images/THISclip_Q.tif"
attributes = "COLOR;MEAN;STD;COUNT;COMPACTNESS;RECTANGULARITY"
# Execute 
TrainMaximumLikelihoodClassifier(inSegRaster, train_features, out_definition, 
                                 in_additional_raster, attributes)

#Accuracy Assement

#Create Points:
# Set local variables
inPointFeatures = "accuracypts.shp"
inRaster = "Q_SVM_classified.tif"
outPointFeatures = "C:/Users/jclav/Project NRS 404/Images/SVM_values_pts.shp"

# Execute ExtractValuesToPoints
ExtractValuesToPoints(inPointFeatures, inRaster, outPointFeatures,
                      "INTERPOLATE", "VALUE_ONLY")


In [12]:
import numpy as np

#SVM: 29 out of 35 correct
a = 29
b = 35
c= (a/b)*100
print("SVM Accuracy is", c)

#ML: 28 out of 35 correct
a1 = 28
b1 = 35
c1= (a1/b1)*100
print("ML Accuracy is", c1)

#RT: 27 out of 35 correct
a2 = 27
b2 = 35
c2= (a2/b2)*100
print("RT Accuracy is", c2)

SVM Accuracy is 82.85714285714286
ML Accuracy is 80.0
RT Accuracy is 77.14285714285715
